<a href="https://colab.research.google.com/github/satyajitghana/TSAI-DeepVision-EVA4.0/blob/master/04_ArchitectureBasics/ArchitectureBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# call upon the python and pytorch gods
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary

In [0]:
"""Net: Our Base Model for MNIST Classification
Attributes
    
"""
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # input - 28x28x1; output - 22x22x32;
        self.conv1 = nn.Sequential(
            # RF - 3x3
            nn.Conv2d(1, 16, 3, bias=False), # 26
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),

            # RF - 5x5
            nn.Conv2d(16, 16, 3, bias=False), # 24
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),

            # RF - 7x7
            nn.Conv2d(16, 32, 3, bias=False), # 22
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.1),
        )

        # translation layer
        # input - 22x22x32; output - 11x11x16
        self.trans1 = nn.Sequential(
            # RF - 7x7
            nn.Conv2d(32, 16, 1, bias=False), # 22
            nn.ReLU(),

            # RF - 14x14
            nn.MaxPool2d(2, 2), # 11
        )
        
        # input - 11x11x16; output - 7x7x16;
        self.conv2 = nn.Sequential(
            # RF - 16x16
            nn.Conv2d(16, 16, 3, bias=False), # 9
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),

            # RF - 18x18
            nn.Conv2d(16, 16, 3, bias=False), # 7
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),
        )

        # input - 7x7x16; output - 5x5x16
        self.conv3 = nn.Sequential(
            # RF - 20x20
            nn.Conv2d(16, 16, 3, padding=1, bias=False), # 5
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),

            # RF - 22x22
            nn.Conv2d(16, 16, 3, bias=False), # 5
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),
        )

        # GAP Layer
        # input - 5x5x16; output - 1x1x10
        self.avg_pool = nn.Sequential(
            # # RF - 22x22
            nn.Conv2d(16, 10, 1, bias=False),
            nn.AvgPool2d(5)
        )

    """forward: performs a forward pass when model(x) is called
    Params
        x: the input data
    Returns
        y: the output of the model
    """
    def forward(self, x):
        x = self.conv1(x)
        x = self.trans1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.avg_pool(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

### Log

- Use Batch Normalization after every convolution
- Add Dropout of 10% on every convolution, except the last layer, don't be too close to the output layer
- Remove the bias terms
- Add a translation layer to converge 32 kernels into 16

In [39]:
# check if cuda is available
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# model the model to the device
model = Net().to(device)
# print the model summary when given input_size
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,304
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
         Dropout2d-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 32, 22, 22]           4,608
             ReLU-10           [-1, 32, 22, 22]               0
      BatchNorm2d-11           [-1, 32, 22, 22]              64
        Dropout2d-12           [-1, 32, 22, 22]               0
           Conv2d-13           [-1, 16, 22, 22]             512
             ReLU-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
# to get consistent results
torch.manual_seed(1)

# batch size
batch_size = 128

# note about pin_memory
# If you load your samples in the Dataset on CPU and would like to push it
# during training to the GPU, you can speed up the host to device transfer by
# enabling pin_memory. This lets your DataLoader allocate the samples in
# page-locked memory, which speeds-up the transfer.

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# train data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

# test data
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
# this automatically selects tqdm for colab_notebook
from tqdm.auto import tqdm, trange

"""train : trains a given model
Params
    model: the model to train
    device: the device to train on
    train_loader: the train data
    optimizer: the optimizer to use
    epoch: epochs to train for

Returns
    None
"""
def train(model, device, train_loader, optimizer, epoch) -> None:
    # set the model in training mode
    model.train()

    pbar = tqdm(train_loader, ncols="80%")

    # run for all the data in train_loader
    for batch_idx, (data, target) in enumerate(pbar):
        # fetch the data and the target value
        data, target = data.to(device), target.to(device)
        # zero out the gradients
        optimizer.zero_grad()
        # forward pass the model with the data
        output = model(data)
        # compute the loss occured
        loss = F.nll_loss(output, target)
        # compute the backward gradients
        loss.backward()
        # update the optimizer params
        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item():.10f} batch_id={batch_idx:05d}')

"""test: tests a given model
Params
    model: the model to test
    device: the device used for testing, cpu/gpu
    test_loader: the test data

Returns
    None
"""
def test(model, device, test_loader) -> None:
    # set the model in evaluation mode
    model.eval()
    # set the loss to zero
    test_loss = 0
    # the number of correct classifications
    correct = 0

    # turn off gradients during the testing
    with torch.no_grad():
        for data, target in test_loader:
            # move the data to device
            data, target = data.to(device), target.to(device)
            # forward pass the model and get the output
            output = model(data)
            # accumulate the loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            # get the index of max log-probability
            pred = output.argmax(dim=1, keepdim=True)
            # increment the correct prediction count if pred is correct
            correct += pred.eq(target.view_as(pred)).sum().item()

    # test loss divided by the total number of items in test dataset
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [42]:
# move the model to the specified device
model = Net().to(device)

# use Stochastic Gradient Descent as the optimizer
# learning_rate=0.02 and momentum=0.7 for SGD
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.7)

# set the number of epochs to train for
N_EPOCH = 20

# run it for N_EPOCH number of times
for epoch in range(1, N_EPOCH+1):
    print('\nEpoch {} : '.format(epoch))
    # train the model
    train(model, device, train_loader, optimizer, epoch)
    # test the model
    test(model, device, test_loader)


Epoch 1 : 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Average loss: 0.0985, Accuracy: 9729/10000 (97.29%)


Epoch 2 : 



Test set: Average loss: 0.0517, Accuracy: 9846/10000 (98.46%)


Epoch 3 : 



Test set: Average loss: 0.0411, Accuracy: 9873/10000 (98.73%)


Epoch 4 : 


...


Test set: Average loss: 0.0201, Accuracy: 9940/10000 (99.40%)


Epoch 17 : 



Test set: Average loss: 0.0209, Accuracy: 9936/10000 (99.36%)


Epoch 18 : 



Test set: Average loss: 0.0193, Accuracy: 9939/10000 (99.39%)


Epoch 19 : 



Test set: Average loss: 0.0206, Accuracy: 9940/10000 (99.40%)


Epoch 20 : 



Test set: Average loss: 0.0209, Accuracy: 9936/10000 (99.36%)

